# Inteligência Artificial - Exercício Árvores

Implemente o A\* para resolver o quebra-cabeças abaixo com duas heurísticas distintas: 
    
    - h1(n) = número de pedras fora do lugar
    - h2(n) = distância total à la Manhattan (horizontal ou vertical) = somatório do deslocamento de cada peça até a posição final  

![puzzle](res/a_estrela_puzzle.png)


As heurísticas solicitadas para resolução desse problema foram: Distância Manhattan e Peças fora do lugar. Sendo assim, definiu-se duas funções para retornar um valor referente a essas heurísticas(dados dois estados para comparação)

In [1]:
from scipy.spatial import distance
import time
MANHATTAN = 1
PIECES = 2

# funcao auxiliar para retornar posicao no jogo
def index2d(index):
    return int(index / 3), index % 3


# calcula o somatorio da distancia manhatan de todas as peças(incluido a casa vazia)
def manhattan(matrix1, matrix2):
    dist = 0
    for i in matrix1:
        dist += distance.cityblock(index2d(matrix1.index(i)), index2d(matrix2.index(i)))
    return dist


# calcula o somatorio de peças fora do ligar(incluido a casa vazia)
def pieces(matrix1, matrix2):
    pieces = 0
    for i in matrix1:
        if matrix1.index(i) != matrix2.index(i):
            pieces += 1
    return pieces


# funcao que aplica a heuristica selecionada
def heuristic(matrix1, matrix2, h):
    if h == MANHATTAN:
        return manhattan(matrix1, matrix2)
    elif h == PIECES:
        return pieces(matrix1, matrix2)

Para facilitar o algoritmo A\* vamos criar uma classe para mapear o estado do nosso problema. Todas as informações que a gente precisa soobre o estado estarão nos parâmetros.

In [2]:
class State:
    def __init__(self, configuration, actual_cost, heuristic_cost, came_from):
        self.configuration = configuration
        self.actual_cost = actual_cost
        self.heuristic_cost = heuristic_cost
        self.total_cost = self.actual_cost + self.heuristic_cost
        self.came_from = came_from

    def __cmp__(self, other):
        return (self.total_cost > other.total_cost) - (self.total_cost < other.total_cost)

No Algoritmo do agente será necessário a função que gerará os filhos do estado atual. O conceito básico dessa função será encontrar a posição da casa vazia e gerar estados mudando as peças vizinhas.

In [3]:
# funca que gera os estados filhos
def generate_children(state, final_configuration, method):
    # lista com os filhos
    children = []

    # encontrando a posicao da casa vazia e as casas vizinhas
    empty_pos = state.configuration.index(0)
    above_pos = empty_pos - 3
    below_pos = empty_pos + 3
    right_pos = empty_pos + 1
    left_pos = empty_pos - 1

    # trocando com a posicao ACIMA caso existir
    if above_pos >= 0:
        new_conf = state.configuration.copy()
        new_conf[empty_pos] = new_conf[above_pos]
        new_conf[above_pos] = 0
        child = State(new_conf, state.actual_cost + 1, heuristic(new_conf, final_configuration, method), state)
        children.append(child)

    # trocando com a posicao ABAIXO caso existir
    if below_pos < len(state.configuration):
        new_conf = state.configuration.copy()
        new_conf[empty_pos] = new_conf[below_pos]
        new_conf[below_pos] = 0
        child = State(new_conf, state.actual_cost + 1, heuristic(new_conf, final_configuration, method), state)
        children.append(child)

    # trocando com a posicao DIREITA
    if right_pos < len(state.configuration) and right_pos != 3 and right_pos != 6:
        new_conf = state.configuration.copy()
        new_conf[empty_pos] = new_conf[right_pos]
        new_conf[right_pos] = 0
        child = State(new_conf, state.actual_cost + 1, heuristic(new_conf, final_configuration, method), state)
        children.append(child)

    # trocando com a posicao ESQUERDA caso existir
    if left_pos >= 0 and left_pos != 2 and left_pos != 5:
        new_conf = state.configuration.copy()
        new_conf[empty_pos] = new_conf[left_pos]
        new_conf[left_pos] = 0
        child = State(new_conf, state.actual_cost + 1, heuristic(new_conf, final_configuration, method), state)
        children.append(child)

    return children

Também será necessário criar uma função para controlar a adição de elementos na fronteira que, se houver uma configuração igual, só mantem o estado com o melhor custo total

In [4]:
# funcao para adicionar na fronteira controlando o valor de custo
def add_to_border(state_list, border):
    already_on_border = False
    for state in state_list:
        for border_state in border:
            if state.configuration == border_state.configuration:
                already_on_border = True
                if state.actual_cost < border_state.actual_cost:
                    border[border.index(border_state)] = state
        if not already_on_border:
            border.append(state)

In [5]:
# funcao para proteger de gerar filhos que já foram visitado
def visited_protect(children, visited):
    children_aux = []
    for c in children:
        for v in visited:
            if c.configuration == v.configuration:
                children_aux.append(c)
    for i in children_aux:
        children.remove(i)
    return children

Funções auxiliares para imprimir os resultados

In [6]:
# funcao que imprime uma configuracao formatada
def print_as_table(configuration):
    table = "{} {} {}\n{} {} {}\n{} {} {}\n=====\n" \
        .format(configuration[0], configuration[1], configuration[2],
                configuration[3], configuration[4], configuration[5],
                configuration[6], configuration[7], configuration[8])
    return table
    

# salva o resultado em um arquivo
def print_result(result, output):
    generations, a_costs, h_costs, solution, total_time, created_nodes, stored_nodes = result
    with open(output, 'w') as file:
        file.write("\nTOTAL DE PASSOS: {}".format(len(solution)))
        file.write("\nGERACOES: {}".format(len(generations)))
        file.write("\nTEMPO (nós criados): {} --> {}ms".format(created_nodes, total_time))
        file.write("\nESPACO (nós armazenados): {}".format(stored_nodes))
        file.write('\nPASSOS PARA A SOLUCAO:\n')
        for r in solution:
            file.write(print_as_table(r.configuration))

Com a estrutura básica criada, agora é necessário modelar o agente que irá fazer a busca A\*. A ideia básica do agente é:
1. adiciona estado na fronteira
2. enquanto fronteira != de vazio
    - seleciona o melhor da fronteira removendo ele da fronteira e adicionando aos "estados visitados"
    - se for estado final, adiciona aos estados e percorre o caminho came_from --> fim
    - se nao for, gera os estados filhos adicionando na fronteira
    - volta 2

In [7]:
def algorithm(initial_state, final_state, heuristic_method):
    start_time = time.time()
    generation = 0
    method = heuristic_method

    # fronteira
    border = []

    # estados visitados
    visited = []

    # representando os estados
    si = initial_state
    sf = final_state

    found_solution = False

    # retornos
    generations = []
    a_costs = []
    h_costs = []
    result = []
    total_time = 0
    created_nodes = 0
    stored_nodes = 0

    # Criando o estado inicial com todos os valores
    start_state = State(si, 0, heuristic(si, sf, method), None)

    # adicionando o estado inicial à fronteira
    border.append(start_state)

    # enquanto a fronteira nao estiver vazia
    while len(border) > 0:
        if len(border) > stored_nodes:
            stored_nodes = len(border)
        generation += 1

        # ordenando para poder obter o melhor a cada iteração
        border.sort(key=lambda state: state.total_cost)

        # selecionando o melhor e colocando na lista dos nós visitados
        actual_state = border[0]
        visited.append(actual_state)

        # removendo o nó visitado
        del (border[0])

        # verificando se o estado já é o final
        if actual_state.heuristic_cost == 0:
            found_solution = True
            break
        else:
            # gerando os estados filhos e adicionado à fronteira
            children = generate_children(actual_state, sf, method)
            children_protected = visited_protect(children,visited)
            children = children_protected
            created_nodes += len(children)
            add_to_border(children, border)
        # print("Geracao: {} \t\t | Custo Atual: {} \t | Custo Heuristica {} "
        #      .format(generation, actual_state.actual_cost, actual_state.heuristic_cost))

        # guardando dados para gráfico
        generations.append(generation)
        a_costs.append(actual_state.actual_cost)
        h_costs.append(actual_state.heuristic_cost)

    if found_solution:
        state = visited.pop()

        while state is not start_state:
            result.append(state)
            state = state.came_from

        # incuindo o no inicial ao resultado
        result.append(start_state)
        # inverntendo a ordem para imprimir o passo correto
        result.reverse()

        end_time = time.time()
        total_time = end_time - start_time

        return generations, a_costs, h_costs, result, total_time, created_nodes, stored_nodes
    else:
        print('Nao encontrou solucao')

Aqui rodamos o algoritmo para cada uma das heuristicas. Os resultados estão em **pieces.out** e **manhattan.out**

In [8]:
initial_state = [7, 2, 4,
                 5, 0, 6,
                 8, 3, 1]

#initial_state = [0, 3, 2,
#                 4, 1, 5,
#                 6, 7, 8]


final_state = [0, 1, 2,
               3, 4, 5,
               6, 7, 8]
  
result_manhattan = algorithm(initial_state, final_state, MANHATTAN)
result_pieces = algorithm(initial_state, final_state, PIECES)

print_result(result_pieces,'pieces.out')
print_result(result_manhattan,'manhattan.out')

### Considerações
**1. Qual o tamanho do espaço de estados?**  
    
O espaço total de busca é igual a $ 9! = 362880 $. Sendo assim a utilizando a h1(peças fora do lugar) percorremos aproximadamente 3.8% do espaço de busca e utilizando a h2(distancia manhattan) percorremos aproximadamente 0.5% do espaço de busca  

**2. Para as duas heurísticas, justificar se são admissíveis e consistentes.**  
    
Uma heurística h(n) é **admissível** se para cada nó n se verifica h(n) ≤ h\*(n), onde h*(n) é o custo real do caminho desde n até ao objectivo. (https://fenix.tecnico.ulisboa.pt/downloadFile/3779572005298/cap4-proc-informada.pdf)    

Uma heurística h(n) é **consistente** (ou monotônica) se para todo nó n e todo sucessor n’ de n gerado por qualquer ação a, o custo estimado de alcançar o objetivo a partir de n não é maior que o custo do passo de se chagar a n’ somado ao custo estimado de alcançar o objetivo a partir de n’. Toda heurística consistente é admissível (http://servicosgasi.sorocaba.unesp.br/assimoes/lectures/iac/downloads/busca_heuristica.pdf)  

Com base nas definições acima, podemos afirmar que ambas as heurísticas são admissíveis pois o custo real de um nó até o objetivo é maior que o custo da heuristica, ou seja, as heurísticas não **sobrestimam**. Podemos afirmar que são consistentes pois ao adicionar na borda de soluções, foi incluido o controle **_visited__protect_** para caso apareça novamente um nó já expandido, mantenha-se o de menor custo. Por elas serem consistentes, podemos reafirmar que são admissíveis também.   

Percebe-se também uma **dominância** da h2(n) sobre a h1(n) pois ela não é tão otimista(está mais perto da realidade) e por consequencia expande menos.    

**3. Compare as soluções encontradas com h1(n) e h2(n)**  
    
A tabela abaixo mostra de forma resumida a diferença em tempo e espaço das duas soluções.  

<div>
<table>
  <tr>
    <th>h1(n) - Peças fora do lugar</th>
    <th>h2(n) - Distancia Manhattan</th>
  </tr>
  <tr>
    <td>
        TOTAL DE PASSOS(profundidade): 27  
        
        GERACOES: 37767  
        
        TEMPO (nós criados): 59457 --> 696.94s  
        
        ESPACO (nós armazenados): 13764  
    </td>
    <td>
        TOTAL DE PASSOS(profundidade): 27  
        
        GERACOES: 3426  
        
        TEMPO (nós criados): 5652 --> 10.29s  
        
        ESPACO (nós armazenados): 1890
    </td>
   </tr>
</table>
</div>



